In [19]:
#import matplotlib.pyplot as plt
#import seaborn as sns

import pandas as pd
import numpy as np
import random as rnd

from sklearn.cross_validation import KFold, cross_val_score

# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier,\
 AdaBoostClassifier, RandomForestClassifier



train_=pd.read_csv('../train_abuse.csv')
validate_=pd.read_csv('../validate_abuse.csv')
#test=pd.read_csv('../testwDSM.csv')

train_.shape, validate_.shape, #test.shape

((1478761, 33), (728346, 33))

In [20]:
train = train_.query('DSMCRIT in [4,9]')
validate = validate_.query('DSMCRIT in [4,9]')
print train['DSMCRIT'].value_counts()
print validate.shape

4    324606
9    108672
Name: DSMCRIT, dtype: int64
(213812, 33)


In [21]:
#train.describe()
train = train.sample(20000)
validate = validate.sample(6000)
train.shape, validate.shape, validate.head(2)

((20000, 33),
 (6000, 33),
         AGE  GENDER  RACE  ETHNIC  MARSTAT  EDUC  EMPLOY  DETNLF  PREG  VET  \
 173863   10       2     2       5        4     3       3       0     2    2   
 331100    8       1     2       5        2     4       3       0     2    2   
 
          ...     FREQ2  FRSTUSE2  SUB3  ROUTE3  FREQ3  FRSTUSE3  NUMSUBS  IDU  \
 173863   ...         1         3     1       0      0         0        2    0   
 331100   ...         1         4     1       0      0         0        2    0   
 
         DSMCRIT  PSYPROB  
 173863        4        1  
 331100        4        2  
 
 [2 rows x 33 columns])

In [22]:
#train = train.query('SUB1 <= 10').query('SUB2 <= 10')
#validate = validate.query('SUB1 <= 10').query('SUB2 <= 10')

drop_list = ['DSMCRIT',  #'NUMSUBS'
             ]

X_train = train.drop(drop_list, axis=1)
Y_train = train["DSMCRIT"]
X_validate = validate.drop(drop_list, axis=1)
Y_validate = validate["DSMCRIT"]
#X_test  = test.drop(drop_list, axis=1)
X_train.shape, X_validate.shape, #X_test.shape



((20000, 32), (6000, 32))

In [23]:
print X_train.columns.tolist()

['AGE', 'GENDER', 'RACE', 'ETHNIC', 'MARSTAT', 'EDUC', 'EMPLOY', 'DETNLF', 'PREG', 'VET', 'LIVARAG', 'ARRESTS', 'SERVSETA', 'METHUSE', 'PSOURCE', 'DETCRIM', 'NOPRIOR', 'SUB1', 'ROUTE1', 'FREQ1', 'FRSTUSE1', 'SUB2', 'ROUTE2', 'FREQ2', 'FRSTUSE2', 'SUB3', 'ROUTE3', 'FREQ3', 'FRSTUSE3', 'NUMSUBS', 'IDU', 'PSYPROB']


In [31]:
from sklearn.feature_selection import SelectKBest, SelectPercentile
from sklearn.feature_selection import f_classif,chi2
#Selector_f = SelectPercentile(f_classif, percentile=25)
Selector_f = SelectKBest(f_classif, k=11)
Selector_f.fit(X_train,Y_train)

X_new = SelectKBest(chi2, k=2).fit_transform(X_train, Y_train)
for n,s in zip(X_train.columns.tolist(),Selector_f.scores_):
     print 'F-score: %3.2ft for feature %s' % (s,n)
print X_new.columns.tolist()

F-score: 1254.78t for feature AGE
F-score: 0.13t for feature GENDER
F-score: 43.03t for feature RACE
F-score: 46.66t for feature ETHNIC
F-score: 209.08t for feature MARSTAT
F-score: 3.58t for feature EDUC
F-score: 431.56t for feature EMPLOY
F-score: 43.37t for feature DETNLF
F-score: 0.59t for feature PREG
F-score: 7.98t for feature VET
F-score: 259.32t for feature LIVARAG
F-score: 0.19t for feature ARRESTS
F-score: 606.90t for feature SERVSETA
F-score: 10.79t for feature METHUSE
F-score: 1566.51t for feature PSOURCE
F-score: 588.16t for feature DETCRIM
F-score: 1199.07t for feature NOPRIOR
F-score: 0.57t for feature SUB1
F-score: 5.31t for feature ROUTE1
F-score: 1579.64t for feature FREQ1
F-score: 297.11t for feature FRSTUSE1
F-score: 116.84t for feature SUB2
F-score: 157.68t for feature ROUTE2
F-score: 346.30t for feature FREQ2
F-score: 226.11t for feature FRSTUSE2
F-score: 106.81t for feature SUB3
F-score: 143.90t for feature ROUTE3
F-score: 187.63t for feature FREQ3
F-score: 217.8

AttributeError: 'numpy.ndarray' object has no attribute 'columns'

In [9]:
#one hot
from sklearn import preprocessing

# 1. INSTANTIATE
enc = preprocessing.OneHotEncoder()

# 2. FIT
enc.fit(X_train)

# 3. Transform
onehotlabels = enc.transform(X_train).toarray()
#onehotlabels.shape
X_train = onehotlabels

onehotlabels = enc.transform(X_validate).toarray()
X_validate = onehotlabels

X_train.shape, X_validate.shape

((20000, 223), (6000, 223))

In [10]:
# Logistic Regression
logreg = LogisticRegression(n_jobs=-1)
logreg.fit(X_train, Y_train)
#Y_pred = logreg.predict(X_test)

l_acc_log = cross_val_score(logreg, X_train, Y_train, cv=5)
acc_log = round(np.mean(l_acc_log), 3)

l_acc_log = ['%.3f' % elem for elem in l_acc_log]
print l_acc_log
print acc_log

/Users/arc/Codes/DM-Lab/dmlab/lib/python2.7/site-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))


['0.811', '0.808', '0.809', '0.813', '0.817']
0.812


In [11]:
# Random Forest (slow)

random_forest = RandomForestClassifier(n_estimators=200, max_depth=20, n_jobs=-1)
random_forest.fit(X_train, Y_train)
#Y_pred = random_forest.predict(X_test)

l_acc_random_forest = cross_val_score(random_forest, X_train, Y_train, cv=5)
acc_random_forest = round(np.mean(l_acc_random_forest), 3)

l_acc_random_forest = ['%.3f' % elem for elem in l_acc_random_forest]
print l_acc_random_forest
print acc_random_forest

['0.810', '0.804', '0.806', '0.809', '0.812']
0.808


In [12]:
# Linear SVC
linear_svc = LinearSVC(C=0.001)
linear_svc.fit(X_train, Y_train)
#Y_pred = linear_svc.predict(X_test)

l_acc_linear_svc = cross_val_score(linear_svc, X_train, Y_train, cv=5)
acc_linear_svc = round(np.mean(l_acc_linear_svc), 3)
l_acc_linear_svc = ['%.3f' % elem for elem in l_acc_linear_svc]
print l_acc_linear_svc
print acc_linear_svc

['0.806', '0.799', '0.810', '0.813', '0.809']
0.807


In [13]:
#Adaboost 
ada_boost = AdaBoostClassifier(random_state=1851)
ada_boost.fit(X_train, Y_train) 

l_acc_ada = cross_val_score(ada_boost, X_train, Y_train, cv=5)
acc_ada = round(np.mean(l_acc_ada), 3)
l_acc_ada = ['%.3f' % elem for elem in l_acc_ada]
print l_acc_ada
print acc_ada


['0.805', '0.804', '0.807', '0.812', '0.818']
0.809


In [14]:
print 'week8-newsplit-score'
models = pd.DataFrame({
    'Model': ['Logistic Regression', 
              'Random Forest','Linear SVC', 'AdaBoost'],
    'Cross Validation': [l_acc_log, 
              l_acc_random_forest, l_acc_linear_svc, l_acc_ada],
    'Cross Validation Mean': [acc_log, 
              acc_random_forest, acc_linear_svc, acc_ada]
    })
models.sort_values(by='Cross Validation Mean', ascending=False)


week8-newsplit-score


,Cross Validation,Cross Validation Mean,Model
0,"[0.811, 0.808, 0.809, 0.813, 0.817]",0.812,Logistic Regression
3,"[0.805, 0.804, 0.807, 0.812, 0.818]",0.809,AdaBoost
1,"[0.810, 0.804, 0.806, 0.809, 0.812]",0.808,Random Forest
2,"[0.806, 0.799, 0.810, 0.813, 0.809]",0.807,Linear SVC


AttributeError: 'numpy.ndarray' object has no attribute 'columns'